In [2]:
# filename: notebooks/explore_users.ipynb

import pandas as pd
from pathlib import Path
import sys

# Add project root to sys.path
project_root = Path().resolve().parent
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

from utils.db_connection import get_db_connection

# Connect to DB
conn = get_db_connection()

# Load tables
users_df = pd.read_sql_query("SELECT * FROM users", conn)
posts_df = pd.read_sql_query("SELECT username FROM posts", conn)
stats_df = pd.read_sql_query("""
    SELECT us.*
    FROM user_stats us
    INNER JOIN (
        SELECT username, MAX(captured_at) AS latest
        FROM user_stats
        GROUP BY username
    ) latest_stats
    ON us.username = latest_stats.username AND us.captured_at = latest_stats.latest
""", conn)

# Count posts per user
post_counts = posts_df['username'].value_counts().reset_index()
post_counts.columns = ['username', 'num_posts']

# Merge summaries
users_summary = users_df.merge(post_counts, on='username', how='left')
users_summary = users_summary.merge(stats_df, on='username', how='left', suffixes=('', '_latest'))

# Clean nulls
users_summary['num_posts'] = users_summary['num_posts'].fillna(0).astype(int)
users_summary['is_banned'] = users_summary['is_banned'].fillna(0).astype(bool)

# Sort by activity
users_summary = users_summary.sort_values(by='num_posts', ascending=False)

# === Summary ===
print("🧑‍💻 Users Summary:")
print(f" - Total users         : {len(users_summary)}")
print(f" - Users with posts    : {(users_summary['num_posts'] > 0).sum()}")
print(f" - Users with 0 posts  : {(users_summary['num_posts'] == 0).sum()}")
print(f" - Users marked banned : {users_summary['is_banned'].sum()}\n")

# === Top Posters ===
print("🏆 Top 10 Users by Post Count:\n")
print(users_summary[['username', 'num_posts']].head(10).to_string(index=False))

# === Top Karma (excluding banned users) ===
print("\n💎 Top 10 Users by Combined Karma:\n")
users_summary['karma_post'] = users_summary.get('karma_post', 0).fillna(0)
users_summary['karma_comment'] = users_summary.get('karma_comment', 0).fillna(0)
users_summary['total_karma'] = users_summary['karma_post'] + users_summary['karma_comment']

top_karma = users_summary[~users_summary['is_banned']].sort_values(by='total_karma', ascending=False)
print(top_karma[['username', 'karma_post', 'karma_comment', 'total_karma']].head(10).to_string(index=False))

# Close DB
conn.close()

🧑‍💻 Users Summary:
 - Total users         : 19404
 - Users with posts    : 8069
 - Users with 0 posts  : 11335
 - Users marked banned : 22

🏆 Top 10 Users by Post Count:

          username  num_posts
         jlundholm        338
     AutoModerator        101
studyingwithraeann         36
         stewtech3         34
No-Engineering9653         34
        Circle_Dot         33
      raquelocasio         30
     Hasekbowstome         30
DisastrousRoll2058         27
         BusyBiegz         27

💎 Top 10 Users by Combined Karma:

           username  karma_post  karma_comment  total_karma
     TotesMessenger   3141592.0      3141592.0    6283184.0
     madpiratebippy     55313.0       605533.0     660846.0
         WtotheSLAM      9580.0       502942.0     512522.0
  classless_classic     19335.0       295616.0     314951.0
          Fritemare      4288.0       232472.0     236760.0
           FoST2015     18076.0       211485.0     229561.0
            myBisL2     11548.0       21474

In [7]:
# filename: notebooks/explore_users.ipynb (continued)

# Reconnect to DB
conn = get_db_connection()

# Input username
username = input("Enter username to explore: ").strip()

if username:
    # Fetch top posts by the user
    user_posts = pd.read_sql_query("""
        SELECT post_id, subreddit_id, title, selftext, score
        FROM posts
        WHERE username = ?
        ORDER BY score DESC
        LIMIT 10
    """, conn, params=(username,))

    # Fetch top comments by the user, joined with posts for subreddit info
    user_comments = pd.read_sql_query("""
        SELECT c.comment_id, p.subreddit_id, c.body, c.score,
               p.title AS parent_post_title,
               pc.body AS parent_comment_body
        FROM comments c
        LEFT JOIN posts p ON c.post_id = p.post_id
        LEFT JOIN comments pc ON c.parent_comment_id = pc.comment_id
        WHERE c.username = ?
        ORDER BY c.score DESC
        LIMIT 10
    """, conn, params=(username,))

    print(f"\nTop Posts by {username}:")
    if not user_posts.empty:
        for idx, row in user_posts.iterrows():
            print(f"{idx+1}. r/{row['subreddit_id']} | Score: {row['score']}")
            print(f"Title: {row['title']}")
            if row['selftext']:
                text = row['selftext'].replace('\n', ' ').strip()
                print(f"Text: {text[:150]}{'...' if len(text) > 150 else ''}")
            print("-" * 40)
    else:
        print("No posts found.")

    print(f"\nTop Comments by {username}:")
    if not user_comments.empty:
        for idx, row in user_comments.iterrows():
            print(f"{idx+1}. r/{row['subreddit_id']} | Score: {row['score']}")
            print(f"Comment: {row['body'][:150]}{'...' if len(row['body']) > 150 else ''}")
            if row['parent_post_title']:
                print(f"Parent Post Title: {row['parent_post_title']}")
            elif row['parent_comment_body']:
                print(f"Parent Comment: {row['parent_comment_body'][:150]}{'...' if len(row['parent_comment_body']) > 150 else ''}")
            print("-" * 40)
    else:
        print("No comments found.")
else:
    print("No username provided.")

conn.close()

Enter username to explore:  TotesMessenger



Top Posts by TotesMessenger:
No posts found.

Top Comments by TotesMessenger:
1. r/3able | Score: 1
Comment: I'm a bot, *bleep*, *bloop*. Someone has linked to this thread from another place on reddit:

- [/r/wgu] [QHT1 Task 2 - Is it really this simple?](htt...
Parent Post Title: QHT1 Task 2 - Is it really this simple?
----------------------------------------
2. r/gnd9a | Score: 1
Comment: I'm a bot, *bleep*, *bloop*. Someone has linked to this thread from another place on reddit:

- [/r/wgu] [Anyone able to help out?](https://www.reddit...
Parent Post Title: How does WGU deal with long breaks?
----------------------------------------
3. r/3pp31 | Score: 0
Comment: I'm a bot, *bleep*, *bloop*. Someone has linked to this thread from another place on reddit:

- [/r/wguit] [What job did you get after graduating?](ht...
Parent Post Title: What job did you get after graduating?
----------------------------------------
